# we want to check how's the overlap between different QTL modalities

In [1]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

from functools import reduce

os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
RNA_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/RNA/output_final/RNA_nominal_CD4_merged.txt", sep = " ")
RNA_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/RNA/output_final/RNA_nominal_CD8_merged.txt", sep = " ")
RNA_permuted_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/RNA/output_final/RNA_permuted_CD4_FDR.txt", sep = " ")
RNA_permuted_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/RNA/output_final/RNA_permuted_CD8_FDR.txt", sep = " ")

In [3]:
ATAC_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_nominal_CD8_merged.txt", sep = " ")
ATAC_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_nominal_CD4_merged.txt", sep = " ")
ATAC_permuted_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_permuted_CD8_FDR.txt", sep = " ")
ATAC_permuted_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_permuted_CD4_FDR.txt", sep = " ")

In [4]:
ins_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD8_merged.txt", sep = " ")
ins_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD4_merged.txt", sep = " ")
ins_permuted_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_permuted_CD8_FDR.txt", sep = " ")
ins_permuted_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_permuted_CD4_FDR.txt", sep = " ")

In [5]:
loop_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD8_merged.txt", sep = " ")
loop_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD4_merged.txt", sep = " ")
loop_permuted_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_permuted_CD8_FDR.txt", sep = " ")
loop_permuted_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_permuted_CD4_FDR.txt", sep = " ")

In [1]:
gtf_annotation_df = pd.read_pickle(f"{base_dir}/gencode_gtf.pickle")
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)

NameError: name 'pd' is not defined

## calculate the overlaps between a QTLs for expression and QTLs for chromatin accessibility of the promoters 

In [8]:
from intervaltree import Interval, IntervalTree
def annotated_peaks(target_ATAC):
    # Create a DataFrame for unique peaks
    df_unique_peaks = target_ATAC[['phe_id', 'phe_chr', 'phe_from', 'phe_to']].drop_duplicates()

    # Initialize an empty DataFrame to store the results
    df_result = pd.DataFrame()

    # Process each chromosome separately
    for chromosome in df_unique_peaks['phe_chr'].unique():
        # Subset data for the current chromosome
        df_peaks_chr = df_unique_peaks[df_unique_peaks['phe_chr'] == chromosome].copy()
        gtf_transcripts_chr = gtf_transcripts[gtf_transcripts['seqname'] == chromosome].copy()
        gtf_transcripts_chr.drop_duplicates(subset=['TSS_start', 'gene_id'], inplace=True)

        # Build interval tree for the current chromosome
        tree = IntervalTree()
        for row in gtf_transcripts_chr.itertuples():
            # Extend TSS_start on 2.5kb in both directions to cover nearby peaks
            tree.add(Interval(row.TSS_start - 2500, row.TSS_start + 2500, row.gene_id))

        # Annotate peaks with gene_id
        rows_list = []
        for i, row in df_peaks_chr.iterrows():
            intervals = tree[row.phe_from:row.phe_to]
            gene_ids = set()  # Store gene_ids to prevent duplicates
            for interval in intervals:  # Iterate through all overlapping intervals
                if interval.data not in gene_ids:
                    new_row = row.to_dict()
                    new_row["gene_id"] = interval.data
                    rows_list.append(new_row)
                    gene_ids.add(interval.data)  # Add gene_id to the set

        df_peaks_chr = pd.DataFrame(rows_list)
        
        df_result = pd.concat([df_result, df_peaks_chr[['phe_id', 'gene_id']]])

    # Now we can merge this back to the original peaks DataFrame
    result = pd.merge(target_ATAC, df_result, how='left', on=['phe_id'])
    return result[result["gene_id"].notnull()]

In [7]:
def get_values_RNA_ATAC_promoter(ref,target):
    # in this case I want the ATAC-peak to be close to the TSS of the gene. The problem of course is that there are multiple TSS per gene.
    annotated_target = annotated_peaks(target)

    # now for each gene that has eQTL, I want to know what directionality the SNP has for the peak that overlaps a TSS
    A_B_merged = ref[(ref["adj_beta_pval"] < 0.1)][["phe_id","var_id","nom_pval","slope"]].merge(annotated_target[["phe_id","var_id","nom_pval","slope","gene_id"]], suffixes = ("_A", "_B"), left_on = ["var_id", "phe_id"], right_on = ["var_id", "gene_id"], how = "left")
    print(len(A_B_merged[A_B_merged["nom_pval_A"] < 0.01])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    print(len(A_B_merged[A_B_merged["nom_pval_B"] < 0.001])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    # fig = px.scatter(A_B_merged, x = "slope_A", y = "slope_B", opacity = 0.2)
    # fig.show()
    df = A_B_merged[A_B_merged["nom_pval_B"] < 0.001]
    # Create concordant mask (both positive or both negative)
    concordant_mask = np.sign(df['slope_A']) == np.sign(df['slope_B'])

    # Create discordant mask (one positive, one negative)
    discordant_mask = np.sign(df['slope_A']) != np.sign(df['slope_B'])

    # Count concordant and discordant rows
    num_concordant = concordant_mask.sum()
    num_discordant = discordant_mask.sum()

    print(f"Number of concordant rows: {num_concordant}")
    print(f"Number of discordant rows: {num_discordant}")
    print(f"ratio: {num_concordant/(num_discordant + num_concordant)}")

In [9]:
get_values_RNA_ATAC_promoter(RNA_permuted_CD8, ATAC_nominal_CD8)

1.0288088642659279
0.1592797783933518
Number of concordant rows: 523
Number of discordant rows: 52
ratio: 0.9095652173913044


In [10]:
get_values_RNA_ATAC_promoter(RNA_permuted_CD4, ATAC_nominal_CD4)

1.025079365079365
0.13936507936507936
Number of concordant rows: 403
Number of discordant rows: 36
ratio: 0.9179954441913439


## calculate the overlaps between a QTLs for expression and QTLs for insulation score of the domain overlapping of the promoters 
also works for loops for all genes contained between the loop anchors

In [11]:
from intervaltree import Interval, IntervalTree
def annotated_ins(target_ins):
    # Create a DataFrame for unique peaks
    df_unique_peaks = target_ins[['phe_id', 'phe_chr', 'phe_from', 'phe_to']].drop_duplicates()

    # Initialize an empty DataFrame to store the results
    df_result = pd.DataFrame()

    # Process each chromosome separately
    for chromosome in df_unique_peaks['phe_chr'].unique():
        # Subset data for the current chromosome
        df_peaks_chr = df_unique_peaks[df_unique_peaks['phe_chr'] == chromosome].copy()
        gtf_transcripts_chr = gtf_transcripts[gtf_transcripts['seqname'] == chromosome].copy()
        gtf_transcripts_chr.drop_duplicates(subset=['TSS_start', 'gene_id'], inplace=True)

        # Build interval tree for the current chromosome
        tree = IntervalTree()
        for row in gtf_transcripts_chr.itertuples():
            tree.add(Interval(row.TSS_start - 1, row.TSS_start + 1, row.gene_id))

        # Annotate peaks with gene_id
        rows_list = []
        for i, row in df_peaks_chr.iterrows():
            intervals = tree[row.phe_from:row.phe_to]
            gene_ids = set()  # Store gene_ids to prevent duplicates
            for interval in intervals:  # Iterate through all overlapping intervals
                if interval.data not in gene_ids:
                    new_row = row.to_dict()
                    new_row["gene_id"] = interval.data
                    rows_list.append(new_row)
                    gene_ids.add(interval.data)  # Add gene_id to the set
        df_peaks_chr = pd.DataFrame(rows_list)
        
        df_result = pd.concat([df_result, df_peaks_chr[['phe_id', 'gene_id']]])

    # Now we can merge this back to the original peaks DataFrame
    result = pd.merge(target_ins, df_result, how='left', on=['phe_id'])
    return result[result["gene_id"].notnull()]

In [12]:
def get_values_RNA_ins_promoter(ref,target):
    # in this case I want the QTL with the insulation score that overlaps the TSS of the gene. The problem of course is that there are multiple TSS per gene.
    annotated_target = annotated_ins(target)

    # now for each gene that has eQTL, I want to know what directionality the SNP has for the peak that overlaps a TSS
    A_B_merged = ref[(ref["adj_beta_pval"] < 0.1)][["phe_id","var_id","nom_pval","slope"]].merge(annotated_target[["phe_id","var_id","nom_pval","slope","gene_id"]], suffixes = ("_A", "_B"), left_on = ["var_id", "phe_id"], right_on = ["var_id", "gene_id"], how = "left")
    print(len(A_B_merged[A_B_merged["nom_pval_A"] < 0.01])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    print(len(A_B_merged[A_B_merged["nom_pval_B"] < 0.001])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    # fig = px.scatter(A_B_merged, x = "slope_A", y = "slope_B", opacity = 0.2)
    # fig.show()
    df = A_B_merged[A_B_merged["nom_pval_B"] < 0.001]
    # Create concordant mask (both positive or both negative)
    concordant_mask = np.sign(df['slope_A']) == np.sign(df['slope_B'])

    # Create discordant mask (one positive, one negative)
    discordant_mask = np.sign(df['slope_A']) != np.sign(df['slope_B'])

    # Count concordant and discordant rows
    num_concordant = concordant_mask.sum()
    num_discordant = discordant_mask.sum()

    print(f"Number of concordant rows: {num_concordant}")
    print(f"Number of discordant rows: {num_discordant}")
    print(f"ratio: {num_concordant/(num_discordant + num_concordant)}")

In [13]:
get_values_RNA_ins_promoter(RNA_permuted_CD8, ins_nominal_CD8)

1.0573407202216067
0.15096952908587258
Number of concordant rows: 465
Number of discordant rows: 80
ratio: 0.8532110091743119


In [14]:
get_values_RNA_ins_promoter(RNA_permuted_CD4, ins_nominal_CD4)

1.0552380952380953
0.14095238095238094
Number of concordant rows: 398
Number of discordant rows: 46
ratio: 0.8963963963963963


In [15]:
get_values_RNA_ins_promoter(RNA_permuted_CD8, loop_nominal_CD8)

1.2728531855955678
0.16786703601108033
Number of concordant rows: 396
Number of discordant rows: 210
ratio: 0.6534653465346535


In [16]:
get_values_RNA_ins_promoter(RNA_permuted_CD4, loop_nominal_CD4)

1.2457142857142858
0.15047619047619049
Number of concordant rows: 330
Number of discordant rows: 144
ratio: 0.6962025316455697


## modified version so that it only considers the promoters overlapping the loop anchors

In [17]:
from intervaltree import Interval, IntervalTree
def annotated_loop_anchors(target_loop):
    # Create a DataFrame for unique peaks
    df_unique_loops = target_loop[['phe_id', 'phe_chr', 'phe_from', 'phe_to']].drop_duplicates()

    # Initialize an empty DataFrame to store the results
    df_result = pd.DataFrame()

    # Process each chromosome separately
    for chromosome in df_unique_loops['phe_chr'].unique():
        # Subset data for the current chromosome
        df_loops_chr = df_unique_loops[df_unique_loops['phe_chr'] == chromosome].copy()
        gtf_transcripts_chr = gtf_transcripts[gtf_transcripts['seqname'] == chromosome].copy()
        gtf_transcripts_chr.drop_duplicates(subset=['TSS_start', 'gene_id'], inplace=True)

        # Build interval tree for the current chromosome
        tree = IntervalTree()
        for row in gtf_transcripts_chr.itertuples():
            tree.add(Interval(row.TSS_start - 2500, row.TSS_start + 2500, row.gene_id))

        # Annotate peaks with gene_id
        rows_list = []
        for i, row in df_loops_chr.iterrows():
            intervals_A = tree[row.phe_from:row.phe_from + 5000]
            intervals_B = tree[row.phe_to - 5000:row.phe_to]
            gene_ids = set()  # Store gene_ids to prevent duplicates
            for interval in intervals_A:  # Iterate through all overlapping intervals of anchor A
                if interval.data not in gene_ids:
                    new_row = row.to_dict()
                    new_row["gene_id"] = interval.data
                    rows_list.append(new_row)
                    gene_ids.add(interval.data)  # Add gene_id to the set
            for interval in intervals_B:  # Iterate through all overlapping intervals of anchor B
                if interval.data not in gene_ids:
                    new_row = row.to_dict()
                    new_row["gene_id"] = interval.data
                    rows_list.append(new_row)
                    gene_ids.add(interval.data)  # Add gene_id to the set
        df_loops_chr = pd.DataFrame(rows_list)
        
        df_result = pd.concat([df_result, df_loops_chr[['phe_id', 'gene_id']]])

    # Now we can merge this back to the original peaks DataFrame
    result = pd.merge(target_loop, df_result, how='left', on=['phe_id'])
    return result[result["gene_id"].notnull()]

In [18]:
def get_values_RNA_loop_anchors_promoter(ref,target):
    # in this case I want the eQTL with the loop anchors
    annotated_target = annotated_loop_anchors(target)

    # now for each gene that has eQTL, I want to know what directionality the SNP has for the peak that overlaps a TSS
    A_B_merged = ref[(ref["adj_beta_pval"] < 0.1)][["phe_id","var_id","nom_pval","slope"]].merge(annotated_target[["phe_id","var_id","nom_pval","slope","gene_id"]], suffixes = ("_A", "_B"), left_on = ["var_id", "phe_id"], right_on = ["var_id", "gene_id"], how = "left")
    print(len(A_B_merged[A_B_merged["nom_pval_A"] < 0.01])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    print(len(A_B_merged[A_B_merged["nom_pval_B"] < 0.001])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    # fig = px.scatter(A_B_merged, x = "slope_A", y = "slope_B", opacity = 0.2)
    # fig.show()
    df = A_B_merged[A_B_merged["nom_pval_B"] < 0.001]
    # Create concordant mask (both positive or both negative)
    concordant_mask = np.sign(df['slope_A']) == np.sign(df['slope_B'])

    # Create discordant mask (one positive, one negative)
    discordant_mask = np.sign(df['slope_A']) != np.sign(df['slope_B'])

    # Count concordant and discordant rows
    num_concordant = concordant_mask.sum()
    num_discordant = discordant_mask.sum()

    print(f"Number of concordant rows: {num_concordant}")
    print(f"Number of discordant rows: {num_discordant}")
    print(f"ratio: {num_concordant/(num_discordant + num_concordant)}")

In [19]:
get_values_RNA_loop_anchors_promoter(RNA_permuted_CD8, loop_nominal_CD8)


1.0493074792243768
0.055401662049861494
Number of concordant rows: 152
Number of discordant rows: 48
ratio: 0.76


In [20]:
get_values_RNA_loop_anchors_promoter(RNA_permuted_CD4, loop_nominal_CD4)

1.0504761904761906
0.051111111111111114
Number of concordant rows: 130
Number of discordant rows: 31
ratio: 0.8074534161490683


## and finally, do the above but with ATAC peaks rather than genes

In [21]:
from intervaltree import Interval, IntervalTree
def annotated_loop_ATAC(target_loop,ref_ATAC):
    # Create a DataFrame for unique peaks
    df_unique_loops = target_loop[['phe_id', 'phe_chr', 'phe_from', 'phe_to']].drop_duplicates()

    # Initialize an empty DataFrame to store the results
    df_result = pd.DataFrame()

    # Process each chromosome separately
    for chromosome in df_unique_loops['phe_chr'].unique():
        # Subset data for the current chromosome
        df_loops_chr = df_unique_loops[df_unique_loops['phe_chr'] == chromosome].copy()
        ref_ATAC_chr = ref_ATAC[ref_ATAC['phe_chr'] == chromosome].copy()
        ref_ATAC_chr.drop_duplicates(subset=['phe_id'], inplace=True)

        # Build interval tree for the current chromosome
        tree = IntervalTree()
        for row in ref_ATAC_chr.itertuples():
            tree.add(Interval(row.phe_from - 5000, row.phe_to + 5000, row.phe_id))

        # Annotate loops with peak ids
        rows_list = []
        for i, row in df_loops_chr.iterrows():
            intervals = tree[row.phe_from:row.phe_to]
            peak_ids = set()  # Store gene_ids to prevent duplicates
            for interval in intervals:  # Iterate through all overlapping intervals
                if interval.data not in peak_ids:
                    new_row = row.to_dict()
                    new_row["peak_ids"] = interval.data
                    rows_list.append(new_row)
                    peak_ids.add(interval.data)  # Add gene_id to the set
        df_loops_chr = pd.DataFrame(rows_list)
        if len(df_loops_chr) > 1:
            df_result = pd.concat([df_result, df_loops_chr[['phe_id', 'peak_ids']]])

    # Now we can merge this back to the original peaks DataFrame
    result = pd.merge(target_loop, df_result, how='left', on=['phe_id'])
    return result[result["peak_ids"].notnull()]

In [22]:
def get_values_ATAC_loop(ref,target):
    annotated_target = annotated_loop_ATAC(target,ref[(ref["adj_beta_pval"] < 0.1)])

    # now for each gene that has eQTL, I want to know what directionality the SNP has for the peak that overlaps a TSS
    A_B_merged = ref[(ref["adj_beta_pval"] < 0.1)][["phe_id","var_id","nom_pval","slope"]].merge(annotated_target[["phe_id","var_id","nom_pval","slope","peak_ids"]], suffixes = ("_A", "_B"), left_on = ["var_id", "phe_id"], right_on = ["var_id", "peak_ids"], how = "left")
    print(len(A_B_merged[A_B_merged["nom_pval_A"] < 0.01])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    print(len(A_B_merged[A_B_merged["nom_pval_B"] < 0.001])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    # fig = px.scatter(A_B_merged, x = "slope_A", y = "slope_B", opacity = 0.2)
    # fig.show()
    df = A_B_merged[A_B_merged["nom_pval_B"] < 0.001]
    # Create concordant mask (both positive or both negative)
    concordant_mask = np.sign(df['slope_A']) == np.sign(df['slope_B'])

    # Create discordant mask (one positive, one negative)
    discordant_mask = np.sign(df['slope_A']) != np.sign(df['slope_B'])

    # Count concordant and discordant rows
    num_concordant = concordant_mask.sum()
    num_discordant = discordant_mask.sum()

    print(f"Number of concordant rows: {num_concordant}")
    print(f"Number of discordant rows: {num_discordant}")
    print(f"ratio: {num_concordant/(num_discordant + num_concordant)}")

In [23]:
get_values_ATAC_loop(ATAC_permuted_CD8, loop_nominal_CD8)

1.3053297199638663
0.2057718822802263
Number of concordant rows: 3230
Number of discordant rows: 1098
ratio: 0.7463031423290203


In [24]:
get_values_ATAC_loop(ATAC_permuted_CD4, loop_nominal_CD4)

1.290559404404922
0.19925550615241444
Number of concordant rows: 2948
Number of discordant rows: 906
ratio: 0.7649195640892579


In [25]:
get_values_ATAC_loop(ATAC_permuted_CD8, ins_nominal_CD8)

1.053392288308848
0.16055721960728378
Number of concordant rows: 3114
Number of discordant rows: 263
ratio: 0.9221202250518211


In [26]:
get_values_ATAC_loop(ATAC_permuted_CD4, ins_nominal_CD4)

1.0521662702926275
0.15205252817702408
Number of concordant rows: 2728
Number of discordant rows: 213
ratio: 0.9275756545392724


In [27]:
from intervaltree import Interval, IntervalTree
def annotated_loop_anchors_ATAC(target_loop,ref_ATAC):
    # Create a DataFrame for unique peaks
    df_unique_loops = target_loop[['phe_id', 'phe_chr', 'phe_from', 'phe_to']].drop_duplicates()

    # Initialize an empty DataFrame to store the results
    df_result = pd.DataFrame()

    # Process each chromosome separately
    for chromosome in df_unique_loops['phe_chr'].unique():
        # Subset data for the current chromosome
        df_loops_chr = df_unique_loops[df_unique_loops['phe_chr'] == chromosome].copy()
        ref_ATAC_chr = ref_ATAC[ref_ATAC['phe_chr'] == chromosome].copy()
        ref_ATAC_chr.drop_duplicates(subset=['phe_id'], inplace=True)

        # Build interval tree for the current chromosome
        tree = IntervalTree()
        for row in ref_ATAC_chr.itertuples():
            tree.add(Interval(row.phe_from - 5000, row.phe_to + 5000, row.phe_id))

        # Annotate loops with peak ids
        rows_list = []
        for i, row in df_loops_chr.iterrows():
            intervals_A = tree[row.phe_from:row.phe_from + 5000]
            intervals_B = tree[row.phe_to - 5000:row.phe_to]
            peak_ids = set()  # Store peak_ids to prevent duplicates
            for interval in intervals_A:  # Iterate through all overlapping intervals of anchor A
                if interval.data not in peak_ids:
                    new_row = row.to_dict()
                    new_row["peak_ids"] = interval.data
                    rows_list.append(new_row)
                    peak_ids.add(interval.data)  # Add peak_ids to the set
            for interval in intervals_B:  # Iterate through all overlapping intervals of anchor B
                if interval.data not in peak_ids:
                    new_row = row.to_dict()
                    new_row["peak_ids"] = interval.data
                    rows_list.append(new_row)
                    peak_ids.add(interval.data)  # Add peak_ids to the set
        df_loops_chr = pd.DataFrame(rows_list)
        if len(df_loops_chr) > 1:
            df_result = pd.concat([df_result, df_loops_chr[['phe_id', 'peak_ids']]])

    # Now we can merge this back to the original peaks DataFrame
    result = pd.merge(target_loop, df_result, how='left', on=['phe_id'])
    return result[result["peak_ids"].notnull()]

In [28]:
def get_values_ATAC_loop_anchors(ref,target):
    annotated_target = annotated_loop_anchors_ATAC(target,ref[(ref["adj_beta_pval"] < 0.1)])

    # now for each gene that has eQTL, I want to know what directionality the SNP has for the peak that overlaps a TSS
    A_B_merged = ref[(ref["adj_beta_pval"] < 0.1)][["phe_id","var_id","nom_pval","slope"]].merge(annotated_target[["phe_id","var_id","nom_pval","slope","peak_ids"]], suffixes = ("_A", "_B"), left_on = ["var_id", "phe_id"], right_on = ["var_id", "peak_ids"], how = "left")
    print(len(A_B_merged[A_B_merged["nom_pval_A"] < 0.01])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    print(len(A_B_merged[A_B_merged["nom_pval_B"] < 0.001])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    # fig = px.scatter(A_B_merged, x = "slope_A", y = "slope_B", opacity = 0.2)
    # fig.show()
    df = A_B_merged[A_B_merged["nom_pval_B"] < 0.001]
    # Create concordant mask (both positive or both negative)
    concordant_mask = np.sign(df['slope_A']) == np.sign(df['slope_B'])

    # Create discordant mask (one positive, one negative)
    discordant_mask = np.sign(df['slope_A']) != np.sign(df['slope_B'])

    # Count concordant and discordant rows
    num_concordant = concordant_mask.sum()
    num_discordant = discordant_mask.sum()

    print(f"Number of concordant rows: {num_concordant}")
    print(f"Number of discordant rows: {num_discordant}")
    print(f"ratio: {num_concordant/(num_discordant + num_concordant)}")

In [29]:
get_values_ATAC_loop_anchors(ATAC_permuted_CD8, loop_nominal_CD8)

1.0583844434935576
0.06737032282603528
Number of concordant rows: 1267
Number of discordant rows: 150
ratio: 0.8941425546930134


In [30]:
get_values_ATAC_loop_anchors(ATAC_permuted_CD4, loop_nominal_CD4)

1.0565091510702098
0.0675731568607176
Number of concordant rows: 1157
Number of discordant rows: 150
ratio: 0.8852333588370314


In [40]:
def get_values_SAME(ref,target):
    A_B_merged = ref[(ref["adj_beta_pval"] < 0.1)][["phe_id","var_id","nom_pval","slope"]].merge(target[["phe_id","var_id","nom_pval","slope"]], suffixes = ("_A", "_B"), on = ["var_id", "phe_id"], how = "left")
    print(len(A_B_merged[A_B_merged["nom_pval_A"] < 0.01])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    print(len(A_B_merged[A_B_merged["nom_pval_B"] < 0.01])/ len(ref[(ref["adj_beta_pval"] < 0.1)]))
    # fig = px.scatter(A_B_merged, x = "slope_A", y = "slope_B", opacity = 0.2)
    # fig.show()
    df = A_B_merged[A_B_merged["nom_pval_B"] < 0.01]
    # Create concordant mask (both positive or both negative)
    concordant_mask = np.sign(df['slope_A']) == np.sign(df['slope_B'])

    # Create discordant mask (one positive, one negative)
    discordant_mask = np.sign(df['slope_A']) != np.sign(df['slope_B'])

    # Count concordant and discordant rows
    num_concordant = concordant_mask.sum()
    num_discordant = discordant_mask.sum()

    print(f"Number of concordant rows: {num_concordant}")
    print(f"Number of discordant rows: {num_discordant}")
    print(f"ratio: {num_concordant/(num_discordant + num_concordant)}")

In [41]:
get_values_SAME(RNA_permuted_CD8,RNA_nominal_CD4)

1.0
0.4695290858725762
Number of concordant rows: 1694
Number of discordant rows: 1
ratio: 0.9994100294985251


In [42]:
get_values_SAME(RNA_permuted_CD4,RNA_nominal_CD8)

1.0
0.4657142857142857
Number of concordant rows: 1464
Number of discordant rows: 3
ratio: 0.9979550102249489


In [43]:
get_values_SAME(ATAC_permuted_CD8,ATAC_nominal_CD4)

1.0
0.40350877192982454
Number of concordant rows: 8475
Number of discordant rows: 12
ratio: 0.9985860728172499


In [44]:
get_values_SAME(ATAC_permuted_CD4,ATAC_nominal_CD8)

1.0
0.4064212594354255
Number of concordant rows: 7846
Number of discordant rows: 15
ratio: 0.9980918458211423


In [45]:
get_values_SAME(ins_permuted_CD8,ins_nominal_CD4)

1.0
0.3812305445601629
Number of concordant rows: 9914
Number of discordant rows: 6
ratio: 0.9993951612903226


In [46]:
get_values_SAME(ins_permuted_CD4,ins_nominal_CD8)

1.0
0.39126053130164873
Number of concordant rows: 9699
Number of discordant rows: 7
ratio: 0.999278796620647


In [47]:
get_values_SAME(loop_permuted_CD8,loop_nominal_CD4)

1.0
0.11887543674331681
Number of concordant rows: 1440
Number of discordant rows: 23
ratio: 0.9842788790157211


In [48]:
get_values_SAME(loop_permuted_CD4,loop_nominal_CD8)

1.0
0.12238033635187581
Number of concordant rows: 1384
Number of discordant rows: 35
ratio: 0.9753347427766033
